In [1]:
from pathlib import Path
import pandas as pd
from dotenv import load_dotenv


def find_project_root(marker: str = ".env") -> Path:
    """Detecta la raíz del proyecto buscando el marcador (.env)."""
    current = Path().resolve()
    for parent in [current] + list(current.parents):
        if (parent / marker).exists():
            return parent
    return current


ROOT_PATH = find_project_root()
load_dotenv(ROOT_PATH / ".env")


def get_data_path(relative_path: str | Path) -> str:
    """Convierte a ruta absoluta (str) y usa *_sample como respaldo si existe."""
    path = Path(relative_path)
    if not path.is_absolute():
        path = ROOT_PATH / path

    if path.exists():
        return str(path)

    sample_path = path.with_name(path.stem + "_sample" + path.suffix)
    if sample_path.exists():
        print(f"⚠️ Usando muestra: {sample_path.name}")
        return str(sample_path)

    raise FileNotFoundError(f"No se encontró el archivo {path} ni su muestra {sample_path}")


print(f"✅ Raíz detectada: {ROOT_PATH}")

✅ Raíz detectada: /home/els4nchez/Videos/Harmeregildo


## 1️⃣ Instalación de Dependencias (si es necesario)

In [2]:
# Instalar TensorFlow/Keras si no está disponible
import subprocess
import sys

try:
    import tensorflow as tf
    print(f"✅ TensorFlow {tf.__version__} ya instalado")
except ImportError:
    print("⚙️ Instalando TensorFlow...")
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'tensorflow', '-q'])
    import tensorflow as tf
    print(f"✅ TensorFlow {tf.__version__} instalado correctamente")

try:
    import keras
    print(f"✅ Keras {keras.__version__} disponible")
except ImportError:
    print("⚙️ Instalando Keras...")
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'keras', '-q'])
    import keras
    print(f"✅ Keras {keras.__version__} instalado")

2026-02-13 16:33:55.711782: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-13 16:33:57.733614: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-13 16:34:02.119412: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


✅ TensorFlow 2.20.0 ya instalado
✅ Keras 3.13.2 disponible


---

## 2️⃣ Importación de Librerías

In [3]:
# Manipulación de datos
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
import json
import warnings
import os
from dotenv import load_dotenv
warnings.filterwarnings('ignore')

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# Preprocesamiento
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Configuración
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Reproducibilidad
np.random.seed(42)
tf.random.set_seed(42)

# Cargar variables de entorno
load_dotenv()

print(f"✅ Librerías importadas correctamente")
print(f"   TensorFlow: {tf.__version__}")
print(f"   Keras: {keras.__version__}")
print(f"   GPU disponible: {tf.config.list_physical_devices('GPU')}")

✅ Librerías importadas correctamente
   TensorFlow: 2.20.0
   Keras: 3.13.2
   GPU disponible: []


2026-02-13 16:34:23.866668: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


---

## 3️⃣ Carga de Datos Integrados

In [4]:
# Configurar rutas
BASE_DIR = ROOT_PATH
DATA_DIR = BASE_DIR / 'data'
UNIFICACION_DIR = BASE_DIR / 'unificacion'
DATA_PROCESADO_DIR = UNIFICACION_DIR / 'datos_procesados'
FIGURAS_DIR = UNIFICACION_DIR / 'figuras'
MODELOS_DIR = UNIFICACION_DIR / 'modelos'

# Crear directorio de modelos si no existe
MODELOS_DIR.mkdir(parents=True, exist_ok=True)

print("📂 Cargando dataset integrado...\n")

# Cargar datos integrados (del notebook 06)
ruta_datos = get_data_path(DATA_PROCESADO_DIR / 'datos_integrados_precios_sentimientos.csv')
df = pd.read_csv(ruta_datos, index_col=0, parse_dates=True)

print(f"✅ Dataset cargado: {df.shape[0]:,} días × {df.shape[1]} variables")
print(f"   Rango: {df.index.min().strftime('%Y-%m-%d')} → {df.index.max().strftime('%Y-%m-%d')}")
print(f"\n📊 Columnas disponibles:")
print(f"   {list(df.columns)}")

📂 Cargando dataset integrado...

✅ Dataset cargado: 2,273 días × 23 variables
   Rango: 2016-01-03 → 2025-10-31

📊 Columnas disponibles:
   ['Open', 'High', 'Low', 'Close', 'Volume', 'Returns', 'Volatility_30', 'outlier_iqr', 'z_score', 'outlier_zscore', 'anomaly_score', 'outlier_iforest', 'outlier_count', 'outlier_consensus', 'sentiment_numeric', 'sentiment', 'sentiment_score', 'num_noticias', 'sentiment_ma7', 'sentiment_ma30', 'sentiment_extreme', 'has_outlier', 'outlier_AND_extreme_sentiment']


---

## 4️⃣ Preparación de Features

In [5]:
# Seleccionar features para los modelos

# Features técnicos base (sin sentimiento)
features_base = [
    'Open', 'High', 'Low', 'Close',
    'Returns',
    'Volatility_30'
]

# Features con sentimiento
features_sentiment = features_base + [
    'sentiment',
    'sentiment_ma7',
    'sentiment_ma30'
]

# Variable objetivo
target = 'Close'

print("📋 CONFIGURACIÓN DE FEATURES")
print("=" * 80)
print(f"\n🔹 Modelo Base (sin sentimiento):")
print(f"   Features: {features_base}")
print(f"   Total: {len(features_base)} variables")

print(f"\n🔸 Modelo con Sentimiento:")
print(f"   Features: {features_sentiment}")
print(f"   Total: {len(features_sentiment)} variables")

print(f"\n🎯 Variable Objetivo: {target}")

# Eliminar filas con NaN en features críticos (usar solo features_sentiment ya que incluye todas las columnas necesarias)
df_clean = df[features_sentiment].dropna()

print(f"\n🧹 Después de limpieza:")
print(f"   Datos válidos: {len(df_clean):,} días")
print(f"   Datos eliminados: {len(df) - len(df_clean):,} días")

📋 CONFIGURACIÓN DE FEATURES

🔹 Modelo Base (sin sentimiento):
   Features: ['Open', 'High', 'Low', 'Close', 'Returns', 'Volatility_30']
   Total: 6 variables

🔸 Modelo con Sentimiento:
   Features: ['Open', 'High', 'Low', 'Close', 'Returns', 'Volatility_30', 'sentiment', 'sentiment_ma7', 'sentiment_ma30']
   Total: 9 variables

🎯 Variable Objetivo: Close

🧹 Después de limpieza:
   Datos válidos: 2,238 días
   Datos eliminados: 35 días


---

## 5️⃣ División Temporal de Datos (Train/Val/Test)

In [6]:
# División temporal: 60% train, 20% validation, 20% test
train_size = int(len(df_clean) * 0.60)
val_size = int(len(df_clean) * 0.20)
test_size = len(df_clean) - train_size - val_size

# Dividir datos
df_train = df_clean.iloc[:train_size]
df_val = df_clean.iloc[train_size:train_size + val_size]
df_test = df_clean.iloc[train_size + val_size:]

print("📊 DIVISIÓN TEMPORAL DE DATOS")
print("=" * 80)
print(f"\n🔹 Train (60%):")
print(f"   Filas: {len(df_train):,}")
print(f"   Rango: {df_train.index.min().strftime('%Y-%m-%d')} → {df_train.index.max().strftime('%Y-%m-%d')}")

print(f"\n🔸 Validation (20%):")
print(f"   Filas: {len(df_val):,}")
print(f"   Rango: {df_val.index.min().strftime('%Y-%m-%d')} → {df_val.index.max().strftime('%Y-%m-%d')}")

print(f"\n🔺 Test (20%):")
print(f"   Filas: {len(df_test):,}")
print(f"   Rango: {df_test.index.min().strftime('%Y-%m-%d')} → {df_test.index.max().strftime('%Y-%m-%d')}")

print(f"\n✅ Total: {len(df_clean):,} días")

# Visualizar división
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_train.index,
    y=df_train['Close'],
    mode='lines',
    name='Train (60%)',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=df_val.index,
    y=df_val['Close'],
    mode='lines',
    name='Validation (20%)',
    line=dict(color='orange')
))

fig.add_trace(go.Scatter(
    x=df_test.index,
    y=df_test['Close'],
    mode='lines',
    name='Test (20%)',
    line=dict(color='green')
))

fig.update_layout(
    title='División Temporal de Datos: Train / Validation / Test',
    xaxis_title='Fecha',
    yaxis_title='Precio de Cierre (USD)',
    template='plotly_white',
    height=500
)

fig.show()

📊 DIVISIÓN TEMPORAL DE DATOS

🔹 Train (60%):
   Filas: 1,342
   Rango: 2016-02-02 → 2022-01-15

🔸 Validation (20%):
   Filas: 447
   Rango: 2022-01-17 → 2024-05-01

🔺 Test (20%):
   Filas: 449
   Rango: 2024-05-02 → 2025-10-14

✅ Total: 2,238 días


---

## 6️⃣ Normalización de Datos

In [7]:
# Crear escaladores (fit solo con datos de entrenamiento)
scaler_base = MinMaxScaler()
scaler_sentiment = MinMaxScaler()

# Escalar features base
X_train_base_scaled = scaler_base.fit_transform(df_train[features_base])
X_val_base_scaled = scaler_base.transform(df_val[features_base])
X_test_base_scaled = scaler_base.transform(df_test[features_base])

# Escalar features con sentimiento
X_train_sent_scaled = scaler_sentiment.fit_transform(df_train[features_sentiment])
X_val_sent_scaled = scaler_sentiment.transform(df_val[features_sentiment])
X_test_sent_scaled = scaler_sentiment.transform(df_test[features_sentiment])

# Escalar target (Close) para desnormalizar predicciones
scaler_target = MinMaxScaler()
y_train_scaled = scaler_target.fit_transform(df_train[[target]])
y_val_scaled = scaler_target.transform(df_val[[target]])
y_test_scaled = scaler_target.transform(df_test[[target]])

print("✅ Normalización completada (MinMaxScaler)")
print(f"\n📊 Rangos de normalización:")
print(f"   Features base: [{X_train_base_scaled.min():.3f}, {X_train_base_scaled.max():.3f}]")
print(f"   Features sentiment: [{X_train_sent_scaled.min():.3f}, {X_train_sent_scaled.max():.3f}]")
print(f"   Target: [{y_train_scaled.min():.3f}, {y_train_scaled.max():.3f}]")

✅ Normalización completada (MinMaxScaler)

📊 Rangos de normalización:
   Features base: [0.000, 1.000]
   Features sentiment: [0.000, 1.000]
   Target: [0.000, 1.000]


---

## 7️⃣ Creación de Secuencias para LSTM

In [8]:
def create_sequences(X, y, seq_length=60):
    """
    Crea secuencias para LSTM.
    
    Args:
        X: Features normalizados (array 2D)
        y: Target normalizado (array 1D o 2D)
        seq_length: Longitud de la secuencia (lookback window)
    
    Returns:
        X_seq: Secuencias de features (3D: samples × seq_length × features)
        y_seq: Targets correspondientes (1D)
    """
    X_seq, y_seq = [], []
    
    for i in range(seq_length, len(X)):
        X_seq.append(X[i - seq_length:i])
        y_seq.append(y[i])
    
    return np.array(X_seq), np.array(y_seq)

# Configuración
SEQ_LENGTH = 60  # 60 días de lookback

print(f"Creando secuencias (lookback={SEQ_LENGTH} días)...\n")

# Crear secuencias para modelo BASE
X_train_base_seq, y_train_seq = create_sequences(X_train_base_scaled, y_train_scaled, SEQ_LENGTH)
X_val_base_seq, y_val_seq = create_sequences(X_val_base_scaled, y_val_scaled, SEQ_LENGTH)
X_test_base_seq, y_test_seq = create_sequences(X_test_base_scaled, y_test_scaled, SEQ_LENGTH)

# Crear secuencias para modelo con SENTIMIENTO
X_train_sent_seq, _ = create_sequences(X_train_sent_scaled, y_train_scaled, SEQ_LENGTH)
X_val_sent_seq, _ = create_sequences(X_val_sent_scaled, y_val_scaled, SEQ_LENGTH)
X_test_sent_seq, _ = create_sequences(X_test_sent_scaled, y_test_scaled, SEQ_LENGTH)

print("✅ Secuencias creadas")
print("\n📊 MODELO BASE (sin sentimiento):")
print(f"   Train: {X_train_base_seq.shape} → {y_train_seq.shape}")
print(f"   Val:   {X_val_base_seq.shape} → {y_val_seq.shape}")
print(f"   Test:  {X_test_base_seq.shape} → {y_test_seq.shape}")

print("\n📊 MODELO CON SENTIMIENTO:")
print(f"   Train: {X_train_sent_seq.shape} → {y_train_seq.shape}")
print(f"   Val:   {X_val_sent_seq.shape} → {y_val_seq.shape}")
print(f"   Test:  {X_test_sent_seq.shape} → {y_test_seq.shape}")

print(f"\n💡 Formato: (samples, seq_length, features)")
print(f"   - samples: número de secuencias")
print(f"   - seq_length: {SEQ_LENGTH} días de historia")
print(f"   - features: {len(features_base)} (base) / {len(features_sentiment)} (sentiment)")

Creando secuencias (lookback=60 días)...



✅ Secuencias creadas

📊 MODELO BASE (sin sentimiento):
   Train: (1282, 60, 6) → (1282, 1)
   Val:   (387, 60, 6) → (387, 1)
   Test:  (389, 60, 6) → (389, 1)

📊 MODELO CON SENTIMIENTO:
   Train: (1282, 60, 9) → (1282, 1)
   Val:   (387, 60, 9) → (387, 1)
   Test:  (389, 60, 9) → (389, 1)

💡 Formato: (samples, seq_length, features)
   - samples: número de secuencias
   - seq_length: 60 días de historia
   - features: 6 (base) / 9 (sentiment)


---

## 8️⃣ Construcción del Modelo LSTM Base (sin sentimiento)

In [9]:
def build_lstm_model(input_shape, model_name="LSTM"):
    """
    Construye modelo LSTM con arquitectura 256-128-64.
    
    Args:
        input_shape: Tupla (seq_length, n_features)
        model_name: Nombre del modelo
    
    Returns:
        Modelo compilado
    """
    model = Sequential(name=model_name)
    
    # Capa de entrada
    model.add(Input(shape=input_shape))
    
    # Primera capa LSTM
    model.add(LSTM(256, return_sequences=True, name='lstm_1'))
    model.add(Dropout(0.2, name='dropout_1'))
    
    # Segunda capa LSTM
    model.add(LSTM(128, return_sequences=True, name='lstm_2'))
    model.add(Dropout(0.2, name='dropout_2'))
    
    # Tercera capa LSTM
    model.add(LSTM(64, return_sequences=False, name='lstm_3'))
    model.add(Dropout(0.2, name='dropout_3'))
    
    # Capa de salida
    model.add(Dense(1, name='output'))
    
    # Compilar modelo
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='mean_squared_error',
        metrics=['mae']
    )
    
    return model

# Construir modelo base
print("Construyendo Modelo LSTM Base (sin sentimiento)...\n")

model_base = build_lstm_model(
    input_shape=(SEQ_LENGTH, len(features_base)),
    model_name="LSTM_Base"
)

print("✅ Modelo Base construido")
print("\n" + "="*80)
model_base.summary()
print("="*80)

Construyendo Modelo LSTM Base (sin sentimiento)...

✅ Modelo Base construido



Model: "LSTM_Base"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 60, 256)        │       269,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 60, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 60, 128)        │       197,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 60, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 515,905 (1.97 MB)

 Trainable params: 515,905 (1.97 MB)

 Non-trainable params: 0 (0.00 B)

---

## 9️⃣ Entrenamiento del Modelo Base

In [10]:
# Configurar callbacks
callbacks_base = [
    EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        filepath=str(MODELOS_DIR / 'lstm_base_best.keras'),
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-6,
        verbose=1
    )
]

print("🚀 Entrenando Modelo Base...\n")
print("Configuración:")
print(f"  - Epochs: 100 (con EarlyStopping)")
print(f"  - Batch size: 32")
print(f"  - Learning rate: 0.001 (con ReduceLROnPlateau)")
print(f"  - Patience: 15 epochs\n")
print("="*80)

# Entrenar modelo
history_base = model_base.fit(
    X_train_base_seq,
    y_train_seq,
    validation_data=(X_val_base_seq, y_val_seq),
    epochs=100,
    batch_size=32,
    callbacks=callbacks_base,
    verbose=1
)

print("\n" + "="*80)
print("✅ Entrenamiento completado")

🚀 Entrenando Modelo Base...

Configuración:
  - Epochs: 100 (con EarlyStopping)
  - Batch size: 32
  - Learning rate: 0.001 (con ReduceLROnPlateau)
  - Patience: 15 epochs

Epoch 1/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - loss: 0.0389 - mae: 0.1280
Epoch 1: val_loss improved from None to 0.00739, saving model to /home/els4nchez/Videos/Harmeregildo/unificacion/modelos/lstm_base_best.keras

Epoch 1: finished saving model to /home/els4nchez/Videos/Harmeregildo/unificacion/modelos/lstm_base_best.keras
41/41 ━━━━━━━━━━━━━━━━━━━━ 13s 206ms/step - loss: 0.0160 - mae: 0.0794 - val_loss: 0.0074 - val_mae: 0.0675 - learning_rate: 0.0010
Epoch 2/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - loss: 0.0035 - mae: 0.0423
Epoch 2: val_loss improved from 0.00739 to 0.00533, saving model to /home/els4nchez/Videos/Harmeregildo/unificacion/modelos/lstm_base_best.keras

Epoch 2: finished saving model to /home/els4nchez/Videos/Harmeregildo/unificacion/modelos/lstm_base_best.keras
41/41 ━━━━━━━━━━━━━━

In [11]:
# Visualizar curvas de aprendizaje
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Loss (MSE)', 'MAE')
)

# Loss
fig.add_trace(
    go.Scatter(y=history_base.history['loss'], mode='lines', name='Train Loss'),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(y=history_base.history['val_loss'], mode='lines', name='Val Loss'),
    row=1, col=1
)

# MAE
fig.add_trace(
    go.Scatter(y=history_base.history['mae'], mode='lines', name='Train MAE'),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(y=history_base.history['val_mae'], mode='lines', name='Val MAE'),
    row=1, col=2
)

fig.update_xaxes(title_text="Epoch", row=1, col=1)
fig.update_xaxes(title_text="Epoch", row=1, col=2)
fig.update_yaxes(title_text="Loss", row=1, col=1)
fig.update_yaxes(title_text="MAE", row=1, col=2)

fig.update_layout(
    title_text="Curvas de Aprendizaje - Modelo Base",
    template='plotly_white',
    height=400
)

fig.show()

print(f"\n📊 Resultados finales:")
print(f"   Train Loss: {history_base.history['loss'][-1]:.6f}")
print(f"   Val Loss: {history_base.history['val_loss'][-1]:.6f}")
print(f"   Train MAE: {history_base.history['mae'][-1]:.6f}")
print(f"   Val MAE: {history_base.history['val_mae'][-1]:.6f}")


📊 Resultados finales:
   Train Loss: 0.001197
   Val Loss: 0.001806
   Train MAE: 0.022969
   Val MAE: 0.029984


---

## 🔟 Construcción y Entrenamiento del Modelo con Sentimiento

In [12]:
# Construir modelo con sentimiento
print("Construyendo Modelo LSTM con Sentimiento...\n")

model_sentiment = build_lstm_model(
    input_shape=(SEQ_LENGTH, len(features_sentiment)),
    model_name="LSTM_Sentiment"
)

print("✅ Modelo con Sentimiento construido")
print("\n" + "="*80)
model_sentiment.summary()
print("="*80)

Construyendo Modelo LSTM con Sentimiento...

✅ Modelo con Sentimiento construido



Model: "LSTM_Sentiment"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 60, 256)        │       272,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 60, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 60, 128)        │       197,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 60, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 518,977 (1.98 MB)

 Trainable params: 518,977 (1.98 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
# Configurar callbacks
callbacks_sentiment = [
    EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        filepath=str(MODELOS_DIR / 'lstm_sentiment_best.keras'),
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-6,
        verbose=1
    )
]

print("🚀 Entrenando Modelo con Sentimiento...\n")
print("="*80)

# Entrenar modelo
history_sentiment = model_sentiment.fit(
    X_train_sent_seq,
    y_train_seq,
    validation_data=(X_val_sent_seq, y_val_seq),
    epochs=100,
    batch_size=32,
    callbacks=callbacks_sentiment,
    verbose=1
)

print("\n" + "="*80)
print("✅ Entrenamiento completado")

🚀 Entrenando Modelo con Sentimiento...

Epoch 1/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step - loss: 0.0347 - mae: 0.1242
Epoch 1: val_loss improved from None to 0.00645, saving model to /home/els4nchez/Videos/Harmeregildo/unificacion/modelos/lstm_sentiment_best.keras

Epoch 1: finished saving model to /home/els4nchez/Videos/Harmeregildo/unificacion/modelos/lstm_sentiment_best.keras
41/41 ━━━━━━━━━━━━━━━━━━━━ 16s 306ms/step - loss: 0.0152 - mae: 0.0818 - val_loss: 0.0065 - val_mae: 0.0635 - learning_rate: 0.0010
Epoch 2/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 274ms/step - loss: 0.0047 - mae: 0.0496
Epoch 2: val_loss improved from 0.00645 to 0.00580, saving model to /home/els4nchez/Videos/Harmeregildo/unificacion/modelos/lstm_sentiment_best.keras

Epoch 2: finished saving model to /home/els4nchez/Videos/Harmeregildo/unificacion/modelos/lstm_sentiment_best.keras
41/41 ━━━━━━━━━━━━━━━━━━━━ 13s 315ms/step - loss: 0.0040 - mae: 0.0464 - val_loss: 0.0058 - val_mae: 0.0614 - learning_rate: 0.0010
E

In [14]:
# Visualizar curvas de aprendizaje
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Loss (MSE)', 'MAE')
)

# Loss
fig.add_trace(
    go.Scatter(y=history_sentiment.history['loss'], mode='lines', name='Train Loss'),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(y=history_sentiment.history['val_loss'], mode='lines', name='Val Loss'),
    row=1, col=1
)

# MAE
fig.add_trace(
    go.Scatter(y=history_sentiment.history['mae'], mode='lines', name='Train MAE'),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(y=history_sentiment.history['val_mae'], mode='lines', name='Val MAE'),
    row=1, col=2
)

fig.update_xaxes(title_text="Epoch", row=1, col=1)
fig.update_xaxes(title_text="Epoch", row=1, col=2)
fig.update_yaxes(title_text="Loss", row=1, col=1)
fig.update_yaxes(title_text="MAE", row=1, col=2)

fig.update_layout(
    title_text="Curvas de Aprendizaje - Modelo con Sentimiento",
    template='plotly_white',
    height=400
)

fig.show()

print(f"\n📊 Resultados finales:")
print(f"   Train Loss: {history_sentiment.history['loss'][-1]:.6f}")
print(f"   Val Loss: {history_sentiment.history['val_loss'][-1]:.6f}")
print(f"   Train MAE: {history_sentiment.history['mae'][-1]:.6f}")
print(f"   Val MAE: {history_sentiment.history['val_mae'][-1]:.6f}")


📊 Resultados finales:
   Train Loss: 0.001528
   Val Loss: 0.003056
   Train MAE: 0.026270
   Val MAE: 0.039060


---

## 1️⃣1️⃣ Evaluación en Conjunto de Test

In [15]:
# Generar predicciones en test set
print("🔮 Generando predicciones en conjunto de test...\n")

# Modelo Base
y_pred_base_scaled = model_base.predict(X_test_base_seq, verbose=0)
y_pred_base = scaler_target.inverse_transform(y_pred_base_scaled)

# Modelo con Sentimiento
y_pred_sent_scaled = model_sentiment.predict(X_test_sent_seq, verbose=0)
y_pred_sent = scaler_target.inverse_transform(y_pred_sent_scaled)

# Valores reales
y_test_real = scaler_target.inverse_transform(y_test_seq)

print("✅ Predicciones generadas")
print(f"   Shape: {y_pred_base.shape}")

🔮 Generando predicciones en conjunto de test...

✅ Predicciones generadas
   Shape: (389, 1)


In [16]:
# Calcular métricas de evaluación
def calculate_metrics(y_true, y_pred, model_name):
    """
    Calcula métricas de evaluación.
    """
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    # MAPE (Mean Absolute Percentage Error)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    return {
        'model': model_name,
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R²': r2,
        'MAPE': mape
    }

# Calcular métricas
metrics_base = calculate_metrics(y_test_real, y_pred_base, 'LSTM Base')
metrics_sent = calculate_metrics(y_test_real, y_pred_sent, 'LSTM + Sentimiento')

# Crear DataFrame de comparación
df_metrics = pd.DataFrame([metrics_base, metrics_sent])

print("📊 COMPARACIÓN DE MODELOS - CONJUNTO DE TEST")
print("="*80)
print(df_metrics.to_string(index=False))
print("="*80)

# Calcular mejora porcentual
mejora_rmse = ((metrics_base['RMSE'] - metrics_sent['RMSE']) / metrics_base['RMSE']) * 100
mejora_mae = ((metrics_base['MAE'] - metrics_sent['MAE']) / metrics_base['MAE']) * 100
mejora_r2 = ((metrics_sent['R²'] - metrics_base['R²']) / abs(metrics_base['R²'])) * 100

print(f"\n🎯 MEJORA CON SENTIMIENTO:")
print(f"   RMSE: {mejora_rmse:+.2f}% ({'mejor' if mejora_rmse > 0 else 'peor'})")
print(f"   MAE:  {mejora_mae:+.2f}% ({'mejor' if mejora_mae > 0 else 'peor'})")
print(f"   R²:   {mejora_r2:+.2f}% ({'mejor' if mejora_r2 > 0 else 'peor'})")

📊 COMPARACIÓN DE MODELOS - CONJUNTO DE TEST
             model           MSE       RMSE        MAE        R²      MAPE
         LSTM Base 274590.418631 524.013758 453.632028 -0.706486 14.135133
LSTM + Sentimiento 308634.561330 555.548883 486.238791 -0.918059 15.202491

🎯 MEJORA CON SENTIMIENTO:
   RMSE: -6.02% (peor)
   MAE:  -7.19% (peor)
   R²:   -29.95% (peor)


---

## 1️⃣2️⃣ Visualización de Predicciones

In [17]:
# Obtener fechas del test set (ajustadas por seq_length)
test_dates = df_test.index[SEQ_LENGTH:]

# Gráfico comparativo
fig = go.Figure()

# Valores reales
fig.add_trace(go.Scatter(
    x=test_dates,
    y=y_test_real.flatten(),
    mode='lines',
    name='Precio Real',
    line=dict(color='black', width=2)
))

# Predicciones modelo base
fig.add_trace(go.Scatter(
    x=test_dates,
    y=y_pred_base.flatten(),
    mode='lines',
    name='LSTM Base',
    line=dict(color='blue', width=1.5, dash='dash')
))

# Predicciones modelo con sentimiento
fig.add_trace(go.Scatter(
    x=test_dates,
    y=y_pred_sent.flatten(),
    mode='lines',
    name='LSTM + Sentimiento',
    line=dict(color='red', width=1.5, dash='dot')
))

fig.update_layout(
    title='🔮 Predicciones vs Valores Reales - Conjunto de Test',
    xaxis_title='Fecha',
    yaxis_title='Precio de Cierre (USD)',
    template='plotly_white',
    height=600,
    hovermode='x unified'
)

fig.show()

In [18]:
# Gráfico de errores (residuos)
errors_base = y_test_real.flatten() - y_pred_base.flatten()
errors_sent = y_test_real.flatten() - y_pred_sent.flatten()

fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Residuos - LSTM Base', 'Residuos - LSTM + Sentimiento'),
    vertical_spacing=0.1
)

# Modelo Base
fig.add_trace(
    go.Scatter(x=test_dates, y=errors_base, mode='lines', name='Residuos Base', line=dict(color='blue')),
    row=1, col=1
)
fig.add_hline(y=0, line_dash="dash", line_color="gray", row=1, col=1)

# Modelo con Sentimiento
fig.add_trace(
    go.Scatter(x=test_dates, y=errors_sent, mode='lines', name='Residuos Sent', line=dict(color='red')),
    row=2, col=1
)
fig.add_hline(y=0, line_dash="dash", line_color="gray", row=2, col=1)

fig.update_xaxes(title_text="Fecha", row=2, col=1)
fig.update_yaxes(title_text="Error (USD)", row=1, col=1)
fig.update_yaxes(title_text="Error (USD)", row=2, col=1)

fig.update_layout(
    title_text="Análisis de Residuos",
    template='plotly_white',
    height=600,
    showlegend=False
)

fig.show()

print(f"\n📊 Estadísticas de errores:")
print(f"\nModelo Base:")
print(f"   Media: {errors_base.mean():.2f} USD")
print(f"   Std: {errors_base.std():.2f} USD")
print(f"   Min: {errors_base.min():.2f} USD")
print(f"   Max: {errors_base.max():.2f} USD")

print(f"\nModelo con Sentimiento:")
print(f"   Media: {errors_sent.mean():.2f} USD")
print(f"   Std: {errors_sent.std():.2f} USD")
print(f"   Min: {errors_sent.min():.2f} USD")
print(f"   Max: {errors_sent.max():.2f} USD")


📊 Estadísticas de errores:

Modelo Base:
   Media: 453.63 USD
   Std: 262.31 USD
   Min: 71.41 USD
   Max: 1308.62 USD

Modelo con Sentimiento:
   Media: 486.24 USD
   Std: 268.71 USD
   Min: 82.54 USD
   Max: 1360.03 USD


---

## 1️⃣3️⃣ Exportación de Resultados

In [19]:
# Exportar predicciones
df_predictions = pd.DataFrame({
    'fecha': test_dates,
    'precio_real': y_test_real.flatten(),
    'pred_base': y_pred_base.flatten(),
    'pred_sentiment': y_pred_sent.flatten(),
    'error_base': errors_base,
    'error_sentiment': errors_sent
})

archivo_pred = DATA_PROCESADO_DIR / 'predicciones_lstm.csv'
df_predictions.to_csv(archivo_pred, index=False)
print(f"✅ Predicciones exportadas: {archivo_pred}")

# Exportar métricas
resultados = {
    'configuracion': {
        'seq_length': SEQ_LENGTH,
        'train_size': len(df_train),
        'val_size': len(df_val),
        'test_size': len(df_test),
        'features_base': features_base,
        'features_sentiment': features_sentiment,
        'arquitectura': '256-128-64',
        'dropout': 0.2,
        'optimizer': 'Adam',
        'learning_rate': 0.001
    },
    'metricas_test': {
        'base': metrics_base,
        'sentiment': metrics_sent,
        'mejora_porcentual': {
            'rmse': float(mejora_rmse),
            'mae': float(mejora_mae),
            'r2': float(mejora_r2)
        }
    },
    'historia_entrenamiento': {
        'base': {
            'epochs': len(history_base.history['loss']),
            'final_train_loss': float(history_base.history['loss'][-1]),
            'final_val_loss': float(history_base.history['val_loss'][-1])
        },
        'sentiment': {
            'epochs': len(history_sentiment.history['loss']),
            'final_train_loss': float(history_sentiment.history['loss'][-1]),
            'final_val_loss': float(history_sentiment.history['val_loss'][-1])
        }
    }
}

archivo_resultados = DATA_PROCESADO_DIR / 'resultados_lstm.json'
with open(archivo_resultados, 'w', encoding='utf-8') as f:
    json.dump(resultados, f, indent=2, ensure_ascii=False)

print(f"✅ Resultados exportados: {archivo_resultados}")

# Guardar modelos finales
model_base.save(MODELOS_DIR / 'lstm_base_final.keras')
model_sentiment.save(MODELOS_DIR / 'lstm_sentiment_final.keras')
print(f"✅ Modelos guardados en: {MODELOS_DIR}")

✅ Predicciones exportadas: /home/els4nchez/Videos/Harmeregildo/unificacion/datos_procesados/predicciones_lstm.csv
✅ Resultados exportados: /home/els4nchez/Videos/Harmeregildo/unificacion/datos_procesados/resultados_lstm.json
✅ Modelos guardados en: /home/els4nchez/Videos/Harmeregildo/unificacion/modelos


---

## 1️⃣4️⃣ Resumen Ejecutivo

In [20]:
print("""
╔════════════════════════════════════════════════════════════════════════╗
║         🤖 RESUMEN EJECUTIVO - MODELO LSTM INTEGRADO                   ║
╠════════════════════════════════════════════════════════════════════════╣
║                                                                        ║
║  ✅ MODELOS ENTRENADOS Y EVALUADOS                                     ║
║                                                                        ║
║  📊 CONFIGURACIÓN:                                                     ║
║                                                                        ║""")

print(f"║  • Arquitectura: 256-128-64 LSTM units                            ║")
print(f"║  • Dropout: 0.2 por capa                                          ║")
print(f"║  • Secuencias: {SEQ_LENGTH} días de lookback                                  ║")
print(f"║  • Train/Val/Test: 60%/20%/20%                                    ║")

print("""
║                                                                        ║
║  📈 RESULTADOS EN TEST SET:                                            ║
║                                                                        ║""")

print(f"║  🔹 MODELO BASE (sin sentimiento):                                 ║")
print(f"║     RMSE: ${metrics_base['RMSE']:7.2f}                                            ║")
print(f"║     MAE:  ${metrics_base['MAE']:7.2f}                                            ║")
print(f"║     R²:   {metrics_base['R²']:7.4f}                                            ║")
print(f"║     MAPE: {metrics_base['MAPE']:6.2f}%                                             ║")

print("""
║                                                                        ║""")

print(f"║  🔸 MODELO CON SENTIMIENTO:                                        ║")
print(f"║     RMSE: ${metrics_sent['RMSE']:7.2f}                                            ║")
print(f"║     MAE:  ${metrics_sent['MAE']:7.2f}                                            ║")
print(f"║     R²:   {metrics_sent['R²']:7.4f}                                            ║")
print(f"║     MAPE: {metrics_sent['MAPE']:6.2f}%                                             ║")

print("""
║                                                                        ║
║  🎯 MEJORA CON SENTIMIENTO:                                            ║
║                                                                        ║""")

print(f"║     RMSE: {mejora_rmse:+6.2f}% ({'MEJOR' if mejora_rmse > 0 else 'PEOR':10s})                            ║")
print(f"║     MAE:  {mejora_mae:+6.2f}% ({'MEJOR' if mejora_mae > 0 else 'PEOR':10s})                            ║")
print(f"║     R²:   {mejora_r2:+6.2f}% ({'MEJOR' if mejora_r2 > 0 else 'PEOR':10s})                            ║")

print("""
║                                                                        ║
║  📂 ARCHIVOS GENERADOS:                                                ║
║                                                                        ║
║  • predicciones_lstm.csv                                              ║
║  • resultados_lstm.json                                               ║
║  • lstm_base_final.keras                                              ║
║  • lstm_sentiment_final.keras                                         ║
║                                                                        ║
║  ➡️  SIGUIENTE PASO:                                                   ║
║     Notebook 08 - Síntesis y Resultados para Paper Científico        ║
║                                                                        ║
╚════════════════════════════════════════════════════════════════════════╝
""")

# Conclusión final
if mejora_rmse > 0 and mejora_mae > 0:
    print("\n✅ CONCLUSIÓN: Los features de sentimiento MEJORAN las predicciones del modelo LSTM.")
    print("   El análisis de sentimientos de noticias tiene valor predictivo para el precio del oro.")
elif mejora_rmse > 2 or mejora_mae > 2:
    print("\n⚠️  CONCLUSIÓN: Los features de sentimiento muestran mejora moderada.")
    print("   El análisis de sentimientos aporta información, pero no es determinante.")
else:
    print("\n❌ CONCLUSIÓN: Los features de sentimiento NO mejoran significativamente las predicciones.")
    print("   Los indicadores técnicos tradicionales son suficientes para este modelo.")


╔════════════════════════════════════════════════════════════════════════╗
║         🤖 RESUMEN EJECUTIVO - MODELO LSTM INTEGRADO                   ║
╠════════════════════════════════════════════════════════════════════════╣
║                                                                        ║
║  ✅ MODELOS ENTRENADOS Y EVALUADOS                                     ║
║                                                                        ║
║  📊 CONFIGURACIÓN:                                                     ║
║                                                                        ║
║  • Arquitectura: 256-128-64 LSTM units                            ║
║  • Dropout: 0.2 por capa                                          ║
║  • Secuencias: 60 días de lookback                                  ║
║  • Train/Val/Test: 60%/20%/20%                                    ║

║                                                                        ║
║  📈 RESULTADOS EN TEST SET:                